In [2]:
from sim import EulerRichardson, G7, G1, solve_ode
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable

class BallisticModel:
    def __init__(self, bc=0.424, rho=0.0765, vs=1125, vx0=2625, vy0=1, x0=0, y0=0.1, g=-32.174, reference=G1, units="imperial", man_x=None, man_v=None, man_y=None):
        if units == "metric":
            self.bc = bc * 0.703  # Convert from lb/in² to kg/m²
            self.rho = rho * 16.0185  # Convert from lb/ft³ to kg/m³
            self.vs = vs * 0.3048  # Convert from ft/s to m/s
            self.g = -9.81  # Gravity in m/s²
        else:
            self.bc = bc * 144 / 32.174
            self.rho = rho / 32.174
            self.vs = vs
            self.g = g

        self.vx0 = vx0
        self.vy0 = vy0
        self.x0 = x0
        self.y0 = y0
        self.reference = reference
        self.units = units

        self.mach_arr = []
        self.cd_arr = []
        self.get_cd = self.interpolate(self.reference)

        self.man_x = np.array(man_x) if man_x is not None else np.array([])
        self.man_v = np.array(man_v) if man_v is not None else np.array([])
        self.man_y = np.array(man_y) if man_y is not None else np.array([])

        self.solve_trajectory()

    def interpolate(self, reference):
        return interp1d(reference[:, 0], reference[:, 1])

    def get_drag(self, speed):
        mach = speed / self.vs
        cd = self.get_cd(mach)
        self.mach_arr.append(mach)

        drag = 0.5 * (1 / self.bc) * self.rho * speed**2 * cd
        self.cd_arr.append(drag)
        return drag

    def projectile(self, t, y):
        x, y, vx, vy = y
        speed = np.sqrt(vx**2 + vy**2)

        drag = self.get_drag(speed)
        u_x = vx / speed if speed != 0 else 0
        u_y = vy / speed if speed != 0 else 0

        ax = -drag * u_x
        ay = self.g - drag * u_y

        return np.array([vx, vy, ax, ay])

    def solve_trajectory(self, t0=0.0, tf=1.4, dt=0.01):
        y0 = np.array([self.x0, self.y0, self.vx0, self.vy0])
        self.t, self.y = solve_ode(self.projectile, (t0, tf), y0, EulerRichardson, self, first_step=dt)

    def generate_table(self):
        distances = np.array([100, 200, 300, 600, 800, 1000])
        drops = []
        drop_errors = []
        speeds = []
        speed_errors = []

        for d in distances:
            index = np.abs(self.y[:, 0] - d).argmin()
            drop = self.y[index, 1] * 100 if self.units == "metric" else self.y[index, 1] * 12
            speed = self.y[index, 2] if self.units == "metric" else self.y[index, 2] * 3.28084

            drops.append(drop)
            speeds.append(speed)

            if self.man_x.size > 0 and d in self.man_x:
                man_index = np.where(self.man_x == d)[0][0]
                drop_errors.append(drop - self.man_y[man_index])
                speed_errors.append(speed - self.man_v[man_index])
            else:
                drop_errors.append(None)
                speed_errors.append(None)

        table = PrettyTable()
        table.field_names = ["Distance (m)" if self.units == "metric" else "Distance (yd)"] + distances.tolist()
        table.add_row(["Drop (cm)" if self.units == "metric" else "Drop (in)"] + drops)
        table.add_row(["Drop Error (cm)" if self.units == "metric" else "Drop Error (in)"] + drop_errors)
        table.add_row(["Speed (m/s)" if self.units == "metric" else "Speed (ft/s)"] + speeds)
        table.add_row(["Speed Error (m/s)" if self.units == "metric" else "Speed Error (ft/s)"] + speed_errors)
        print(table)

    def plot_reference(self):
        plt.figure(figsize=(5, 3))
        plt.plot(self.reference[:, 0], self.reference[:, 1], 'black', label='Reference Curve')
        plt.axvline(x=1.0, color='k', linestyle='--', label='Speed of Sound')
        plt.ylabel('Cd')
        plt.xlabel('Mach Number')
        plt.title('Drag Coefficient vs. Mach Number')
        plt.legend()
        plt.grid(True)
        plt.show()


In [ ]:
b = BallisticModel(bc=1.05, vx0=2815)
y0 = np.array([b.x0, b.y0, b.vx0, b.vy0])
dt = 0.01
t0 = 0.0
tf = 1.4

t, y = solve_ode(projectile, (t0, tf), y0, EulerRichardson, b, first_step=dt)
vx = y[:, 2]
vy = y[:, 3]
x = y[:, 0]
y = y[:, 1]
'''
man_x = np.array([0, 100, 200, 300, 600, 800, 1000])
man_v = np.array([2625, 2473, 2325, 2183, 1786, 1545, 1332])
man_y = np.array([0, 0, -4, -15, -94, -200, -365])
'''

man_x = np.array([0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])  # Range in yards
man_v = np.array([2815, 2727, 2641, 2557, 2474, 2393, 2314, 2235, 2159, 2083, 2010])  # Velocity in ft/s
man_y = np.array([-1.5,  0.0, -3.2, -11.4, -24.9, -44.0, -69.3, -101.0, -139.7, -185.9, -240.1])

